<a href="https://colab.research.google.com/github/vaibhav016/Quantum_speech/blob/main/15_jan_circuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd ..

/


In [2]:
mkdir dataset


In [3]:
cd dataset

/dataset


In [4]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz

--2022-01-15 23:29:47--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.107.128, 2607:f8b0:400c:c32::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.107.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   188MB/s    in 8.1s    

2022-01-15 23:29:55 (176 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]



In [5]:
!tar -xf speech_commands_v0.01.tar.gz

In [6]:
pip install pennylane

     |████████████████████████████████| 756 kB 5.0 MB/s 
     |████████████████████████████████| 305 kB 71.1 MB/s 
     |████████████████████████████████| 108 kB 78.2 MB/s 


In [7]:
cd ..

/


In [8]:
import pennylane as qml
from pennylane import numpy as np
import pennylane as qml
import tensorflow as tf
import sklearn.datasets
import os
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile
from tqdm import tqdm
import warnings
import tensorflow as tf


from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop, SGD

from sklearn.model_selection import train_test_split
wires = 8  
dev = qml.device('default.qubit', wires=wires)
train_audio_path = 'dataset/'
sr=16000


In [9]:
# Generating mel 
def gen_mel(labels, train_audio_path, sr):
    all_wave = []
    all_label = []
    for label in tqdm(labels):
        waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
        for num, wav in enumerate(waves, 0):
            y, _ = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = sr)
            if(len(y)== sr) :
                mel_feat = librosa.feature.melspectrogram(y, sr=sr, n_fft=1024, hop_length=1024, power=1.0, n_mels=16, fmin=40.0, fmax=sr/2)
                all_wave.append(np.reshape(mel_feat,-1))
                all_label.append(label)

    return all_wave, all_label

labels = [
    'left', 'go', 'yes', 'down', 'up', 'on', 'right', 'no', 'off', 'stop',
]

train_audio_path = 'dataset/'

all_wave, all_label = gen_mel(labels, train_audio_path, sr)

100%|██████████| 10/10 [00:24<00:00,  2.43s/it]


In [10]:
# making training data 
label_enconder = LabelEncoder()
y = label_enconder.fit_transform(all_label)
classes = list(label_enconder.classes_)
y = keras.utils.to_categorical(y, num_classes=len(labels))

# input_mel = tf.convert_to_tensor(all_wave, dtype=tf.float32)
# print(input_mel.shape)

# y_tf = tf.convert_to_tensor(y, dtype=tf.float32)
# print(y.shape)

x_train, x_valid, y_train, y_valid = train_test_split(np.array(all_wave),np.array(y),test_size = 0.2,random_state=777,shuffle=True)


In [11]:
def conv_alternate_inner_circuit(weights, i, j): 
#     print("inside alternate_circuit, weights shape", weights, weights.shape)
    qml.CNOT(wires=[j, i])
    qml.RZ(weights[0], wires = i)
    qml.RY(weights[1], wires = j)
    qml.CNOT(wires=[i, j])
    qml.RY(weights[2], wires = j)
    qml.CNOT(wires=[j, i])

def conv_circuit_join(weights):
#     print("inside conv_circuit, weights shape", weights, weights.shape)
    for i in range(1,7,2):
        conv_alternate_inner_circuit(weights[i], i, i+1)
    for i in range(0,7,2):
        conv_alternate_inner_circuit(weights[i], i, i+1)

def quantum_pooling():
    # both these cnot and S are non-parametric gates, so no weights.
    for i in range(0,7,2):
        qml.CNOT(wires=[i+1, i])
        qml.S(wires=i)

In [12]:
@qml.qnode(dev)
def qnode(inputs, weights_conv_0,weights_conv_1, weights_conv_2, weights_conv_3):
    qml.AmplitudeEmbedding(inputs, wires=range(8), pad_with=0.,normalize=True) # state preparation. 
    conv_circuit_join(weights_conv_0)  
    quantum_pooling()
    conv_alternate_inner_circuit(weights_conv_1, 0, 2)
    conv_alternate_inner_circuit(weights_conv_2,4, 6)
    conv_alternate_inner_circuit(weights_conv_3,2, 6)
    
    #Last pooling 
    qml.CNOT(wires=[2, 0])
    qml.S(wires=0)
    
    qml.CNOT(wires=[6, 4])
    qml.S(wires=4)
    
    # Final fully connected layer for binary classification
    qml.CNOT(wires=[0, 4])
    
    m = [qml.expval(qml.PauliZ(i)) for i in range(8)]
    return m

In [13]:
def create_quantum_model():
    weight_shapes = {"weights_conv_0": (8,3),"weights_conv_1":[3],"weights_conv_2":[3]
                ,"weights_conv_3":[3]}
    qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=8)
    clayer2 = keras.layers.Dense(10, activation="softmax")
    model = tf.keras.models.Sequential([qlayer, clayer2])
    
    model.compile(
        optimizer=SGD(lr=0.01, momentum=0.9),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

    
model_qcnn = create_quantum_model()




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [27]:
checkpoint_path = "content/drive/MyDrive/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'content/drive/MyDrive/training_1/cp.ckpt'

None


In [28]:

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True, save_freq='epoch',
                                                 verbose=1)



In [29]:
model_qcnn.load_weights(latest)

In [ ]:
dx, dy = x_train[:20], y_train[:20]
model_qcnn.fit(
    x_train,
    y_train,
    batch_size=20,
    epochs=10,
    verbose=1,
callbacks=[cp_callback])


Epoch 1/10
  9/853 [..............................] - ETA: 1:14:31 - loss: 1.7909 - accuracy: 0.2944